In [ ]:
!pip install opendatasets --quiet --upgrade
import opendatasets as od
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib
import os
%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)
sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (10, 6)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

In [ ]:
#od.download('https://www.kaggle.com/fivethirtyeight/uber-pickups-in-new-york-city')

In [ ]:
df_1 = pd.read_csv('../input/uber-pickups-in-new-york-city/uber-raw-data-apr14.csv')
df_2 = pd.read_csv('../input/uber-pickups-in-new-york-city/uber-raw-data-aug14.csv')
df_4 = pd.read_csv('../input/uber-pickups-in-new-york-city/uber-raw-data-jul14.csv')
df_5 = pd.read_csv('../input/uber-pickups-in-new-york-city/uber-raw-data-jun14.csv')
df_6 = pd.read_csv('../input/uber-pickups-in-new-york-city/uber-raw-data-may14.csv')
df_7 = pd.read_csv('../input/uber-pickups-in-new-york-city/uber-raw-data-sep14.csv')

In [ ]:
df = pd.concat([df_1,df_2,df_4,df_5,df_6,df_7])

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df['Date/Time'] = pd.to_datetime(df['Date/Time'], format="%m/%d/%Y %H:%M:%S")

In [ ]:
df.info()

In [ ]:
df['weekday'] = df['Date/Time'].dt.day_name()

In [ ]:
df['day'] =  df['Date/Time'].dt.day

In [ ]:
df['minute'] =  df['Date/Time'].dt.minute
df['hour'] =  df['Date/Time'].dt.hour
df['month'] =  df['Date/Time'].dt.month

In [ ]:
df.head()

In [ ]:
df['weekday'].value_counts()

In [ ]:
import plotly.express as px

In [ ]:
px.bar(
    x= df['weekday'].value_counts().index,
    y=df['weekday'].value_counts()
)

now we can see that rides is definitly high on tuesday

In [ ]:
plt.figure(figsize=(20,10))
for i, month in enumerate(df['month'].unique()):
  plt.subplot(3,2,i+1)
  df[df['month']==month]['hour'].hist()
  plt.title('Rides hours in month {m}'.format(m = month))

In [ ]:
df['month'].value_counts()

# **Analysis of which month has max rides (hour per month)**

In [ ]:
df.groupby('month')['hour'].count()

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(df.groupby('month')['hour'].count().index, df.groupby('month')['hour'].count())
plt.xlabel('Month')
plt.ylabel('no of hours')
plt.title('Rides hours per month')

**then september (9) has the maximum rides**

# **Analysis of journey of each day**

In [ ]:
df['day'].value_counts()

In [ ]:
plt.figure(figsize=(15,8))
plt.hist(x = df['day'],bins=30, rwidth=0.8, range=(0.5,30.5))
plt.xlabel('date of the month')
plt.ylabel('total journeys')
plt.title('total journeys per each day in month')

In [ ]:
plt.figure(figsize=(20,10))
for i, month in enumerate(df['month'].unique()):
  plt.subplot(3,2,i+1)
  df[df['month']==month]['hour'].hist()
  plt.title('Rush hours in month {m}'.format(m = month))

# **Analysis of rides in hours**

In [ ]:
df.head()

In [ ]:
sns.pointplot(x='hour',y='Lat', hue='weekday',data=df)

# **Analyse which base number popular by month**
The Base codes are for the following Uber bases:

- B02512 : Unter
- B02598 : Hinter
- B02617 : Weiter
- B02682 : Schmecken
- B02764 : Danach-NY
- B02765 : Grun
- B02835 : Dreist
- B02836 : Drinnen

In [ ]:
MonthBase = df.groupby(['Base','month'])['Date/Time'].count().reset_index()
MonthBase

In [ ]:
plt.figure(figsize=(12,8))
plt.title('MOst popular base in month')
sns.lineplot(x='month',y='Date/Time',hue='Base',data=MonthBase)

In [ ]:
#heatmap by hour and weekday
def count(rows):
  return len(rows)
df.groupby(['weekday','hour']).apply(count)

which means at every hour of that day there was (number) of orders

In [ ]:
weekday_hour = df.groupby(['weekday','hour']).apply(count)

In [ ]:
weekday_hour.unstack()

In [ ]:
df_uber_foil = pd.read_csv('../input/uber-pickups-in-new-york-city/Uber-Jan-Feb-FOIL.csv')

In [ ]:
df_uber_foil.head()

# **Analyse which base Number has most vehicles**

In [ ]:
df_uber_foil.groupby(['dispatching_base_number','active_vehicles']).count().reset_index()

In [ ]:
sns.barplot(x='dispatching_base_number', y='active_vehicles',data=df_uber_foil)

# **Analyze which base number has most trips**

In [ ]:
sns.barplot(x='dispatching_base_number', y='trips',data=df_uber_foil)

# **Analyze that how average trip/vechile affect(inc/dec) within date and base number**

In [ ]:
df_uber_foil['trip/vechile'] = df_uber_foil['trips'] / df_uber_foil['active_vehicles']

In [ ]:
plt.figure(figsize=(15,5))
df_uber_foil.set_index('date').groupby(['dispatching_base_number'])['trip/vechile'].plot()
plt.legend()